In [8]:
import re
import string

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import transformers
from transformers import BertTokenizer
from transformers import TFAutoModel
from transformers import AutoTokenizer

In [2]:
import os 

In [3]:
os.listdir()

['.ipynb_checkpoints',
 'app.py',
 'cardiffnlp',
 'Dockerfile',
 'Model Test.ipynb',
 'model.py',
 'prepare_data.py',
 'requirements.txt',
 '__pycache__']

In [4]:
df = pd.read_csv('../data/process/twitter_train_cleaned.csv')

In [5]:
df.head()

,ID,origin,sentiment,tweet
0,2403,Borderlands,0,"Rock-Hard La Varlope, RARE & POWERFUL, HANDSOM..."
1,2408,Borderlands,0,Check out this epic streamer!.
2,2409,Borderlands,0,Blaming Sight for Tardiness! A little bit of b...
3,2411,Borderlands,0,. . [
4,2414,Borderlands,0,One of our own @ProfZeroo is live w/ @borderla...


In [27]:
seq_len = 256
batch_size = 16
num_samples = len(df)
model_name = 'cardiffnlp/twitter-roberta-base-sentiment'

In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

404 Client Error: Not Found for url: https://huggingface.co/cardiffnlps/twitter-roberta-base-sentiment/resolve/main/config.json


OSError: Can't load config for 'cardiffnlps/twitter-roberta-base-sentiment'. Make sure that:

- 'cardiffnlps/twitter-roberta-base-sentiment' is a correct model identifier listed on 'https://huggingface.co/models'
  (make sure 'cardiffnlps/twitter-roberta-base-sentiment' is not a path to a local directory with something else, in that case)

- or 'cardiffnlps/twitter-roberta-base-sentiment' is the correct path to a directory containing a config.json file



In [19]:
train_tokens = tokenizer(df['tweet'].tolist(), max_length=seq_len, 
                         truncation=True, padding='max_length', 
                         add_special_tokens=True, return_tensors='np')

y_train = df['sentiment'].values
labels = np.zeros((num_samples, y_train.max() + 1))
labels[np.arange(num_samples), y_train] = 1

dataset = tf.data.Dataset.from_tensor_slices((train_tokens['input_ids'], train_tokens['attention_mask'], labels))

def map_func(input_ids, masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': masks
    }, labels

dataset = dataset.map(map_func)
dataset = dataset.shuffle(10000).batch(batch_size=batch_size, drop_remainder=True)

split = 0.7
size = int((train_tokens['input_ids'].shape[0] // batch_size) * split)

train_ds = dataset.take(size)
val_ds = dataset.skip(size)

NameError: name 'tokenizer' is not defined

In [30]:
def model():
    """
    Loading the pre-trained model
    """
    task='sentiment'
    MODEL = f"cardiffnlp/twitter-roberta-base-{'sentiment'}"
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    labels = []
    mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
    with urllib.request.urlopen(mapping_link) as f:
        html = f.read().decode('utf-8').split("\n")
        csvreader = csv.reader(html, delimiter='\t')
    labels = [row[1] for row in csvreader if len(row) > 1]
    model = AutoModelForSequenceClassification.from_pretrained(MODEL)
    model.save_pretrained(MODEL)
    return model, tokenizer, labels

In [31]:
model, tokenizer,labels = model()

404 Client Error: Not Found for url: https://huggingface.co/cardiffnlps/twitter-roberta-base-sentiment/resolve/main/config.json


OSError: Can't load config for 'cardiffnlps/twitter-roberta-base-sentiment'. Make sure that:

- 'cardiffnlps/twitter-roberta-base-sentiment' is a correct model identifier listed on 'https://huggingface.co/models'
  (make sure 'cardiffnlps/twitter-roberta-base-sentiment' is not a path to a local directory with something else, in that case)

- or 'cardiffnlps/twitter-roberta-base-sentiment' is the correct path to a directory containing a config.json file



In [36]:
import torch

In [43]:
os.path.join('./',os.listdir('./cardiffnlp/twitter-roberta-base-sentiment/'))

TypeError: join() argument must be str or bytes, not 'list'

In [41]:
model = torch.load(os.path.join('./',os.listdir('./cardiffnlp/twitter-roberta-base-sentiment/')))
model.eval()

TypeError: join() argument must be str or bytes, not 'list'